# [Lesson 08: Profiling Python Code](https://machinelearningmastery.com/profiling-python-code/)

Profiling is a technique to figure out how time is spent in a program. With
these statistics, we can find the “hot spot” of a program and think about ways
of improvement. Sometimes, a hot spot in an unexpected location may hint at a
bug in the program as well.

In this tutorial, we will see how we can use the profiling facility in Python.
Specifically, you will see:

*    How we can compare small code fragments using the timeit module
*    How we can profile the entire program using the cProfile module
*    How we can invoke a profiler inside an existing program
*    What the profiler cannot do

## Tutorial Overview

This tutorial is in four parts; they are:

*    Profiling small fragments
*    The profile module
*    Using profiler inside code
*    Caveats

## Profiling Small Fragments

When you are asked about the different ways of doing the same thing in Python,
one perspective is to check which one is more efficient. In Python’s standard
library, we have the timeit module that allows us to do some simple profiling.

For example, to concatenate many short strings, we can use the `join()` function
from strings or the + operator. So, how do we know which is faster? Consider the
following Python code:


In [ ]:
longstr = ""
for x in range(1000):
  longstr += str(x)

This will produce a long string 012345.... in the variable longstr. An
alternative way to write this is:

In [ ]:
longstr = "".join([str(x) for x in range(1000)])

To compare the two, we can do the following at the command line:

In [ ]:

!python -m timeit 'longstr=""' 'for x in range(1000): longstr += str(x)'
!python -m timeit '"".join([str(x) for x in range(1000)])'

The above commands are to load the timeit module and pass on a single line of
code for measurement. In the first case, we have two lines of statements, and
they are passed on to the timeit module as two separate arguments. In the same
rationale, the first command can also be presented as three lines of statements
(by breaking the for-loop into two lines), but the indentation of each line
needs to be quoted correctly:

In [ ]:
!python -m timeit 'longstr=""' 'for x in range(1000):' ' longstr += str(x)'

The output of timeit is to find the best performance among multiple runs
(default to be 5). Each run is to run the provided statements a few times (which
is dynamically determined). The time is reported as the average to execute the
statements once in the best run.

While it is true that the join function is faster than the + operator for string
concatenation, the timing above is not a fair comparison. It is because we use
str(x) to make short strings on the fly during the loop. The better way to do
this is the following:

In [ ]:
!python -m timeit -s 'strings = [str(x) for x in range(1000)]' 'longstr=""' 'for x in strings:' ' longstr += str(x)'
!python -m timeit -s 'strings = [str(x) for x in range(1000)]' '"".join(strings)'

The -s option allows us to provide the “setup” code, which is executed before
the profiling and not timed. In the above, we create the list of short strings
before starting the loop. Hence the time to create those strings is not measured
in the “per loop” timing. The above shows that the join() function is two orders
of magnitude faster than the + operator. The more common use of the -s option is
to import the libraries. For example, we can compare the square root function
from Python’s math module from NumPy and use the exponential operator ** as
follows:

In [ ]:
!python -m timeit '[x**0.5 for x in range(1000)]'
!python -m timeit -s 'from math import sqrt' '[sqrt(x) for x in range(1000)]'
!python -m timeit -s 'from numpy import sqrt' '[sqrt(x) for x in range(1000)]'

The above produces the following measurement, which we see that math.sqrt() is
fastest while numpy.sqrt() is slowest in this particular example:

If you wonder why NumPy is the slowest, it is because NumPy is optimized for
arrays. You will see its exceptional speed in the following alternative:

In [ ]:
!python -m timeit -s 'import numpy as np; x=np.array(range(1000))' 'np.sqrt(x)'

If you prefer, you can also run timeit in Python code. For example, the
following will be similar to the above but give you the total raw timing for
each run:

In [ ]:
import timeit
measurements = timeit.repeat('[x**0.5 for x in range(1000)]', number=10000)
print(measurements)

In the above, each run is to execute the statement 10,000 times; the result is as
follows. You can see the result of roughly 71 usec per loop in the best run:

## The Profile Module

Focusing on a statement or two for performance is from a microscopic
perspective. Chances are, we have a long program and want to see what is causing
it to run slow. That happens before we can consider alternative statements or
algorithms.

A program running slow can generally be due to two reasons: A part is running
slow, or a part is running too many times, adding up and taking too much time.
We call these “performance hogs” the hot spot. Let’s look at an example.
Consider the following program that uses a hill-climbing algorithm to find
hyperparameters for a perceptron model:


In [1]:
# manually search perceptron hyperparameters for binary classification
from numpy import mean
from numpy.random import randn
from numpy.random import rand
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import Perceptron

# objective function
def objective(X, y, cfg):
	# unpack config
	eta, alpha = cfg
	# define model
	model = Perceptron(penalty='elasticnet', alpha=alpha, eta0=eta)
	# define evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# evaluate model
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	# calculate mean accuracy
	result = mean(scores)
	return result

# take a step in the search space
def step(cfg, step_size):
	# unpack the configuration
	eta, alpha = cfg
	# step eta
	new_eta = eta + randn() * step_size
	# check the bounds of eta
	if new_eta <= 0.0:
		new_eta = 1e-8
	if new_eta > 1.0:
		new_eta = 1.0
	# step alpha
	new_alpha = alpha + randn() * step_size
	# check the bounds of alpha
	if new_alpha < 0.0:
		new_alpha = 0.0
	# return the new configuration
	return [new_eta, new_alpha]

# hill climbing local search algorithm
def hillclimbing(X, y, objective, n_iter, step_size):
	# starting point for the search
	solution = [rand(), rand()]
	# evaluate the initial point
	solution_eval = objective(X, y, solution)
	# run the hill climb
	for i in range(n_iter):
		# take a step
		candidate = step(solution, step_size)
		# evaluate candidate point
		candidate_eval = objective(X, y, candidate)
		# check if we should keep the new point
		if candidate_eval >= solution_eval:
			# store the new point
			solution, solution_eval = candidate, candidate_eval
			# report progress
			print('>%d, cfg=%s %.5f' % (i, solution, solution_eval))
	return [solution, solution_eval]

# define dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=2, n_redundant=1, random_state=1)
# define the total iterations
n_iter = 100
# step size in the search space
step_size = 0.1
# perform the hill climbing search
cfg, score = hillclimbing(X, y, objective, n_iter, step_size)
print('Done!')
print('cfg=%s: Mean Accuracy: %f' % (cfg, score))

The normal output of the program will be printed first, and then the profiler’s
statistics will be printed. From the first row, we see that the function
`objective()` in our program has run 101 times and took 4.89 seconds. But these
4.89 seconds are mostly spent on the functions it called, which the total time
spent on that function is merely 0.001 seconds. The functions from dependent
modules are also profiled. Hence you see a lot of NumPy functions above too.

The above output is long and may not be useful to you as it can be difficult to
tell which function is the hot spot. Indeed we can sort the above output. For
example, to see which function is called the most number of times, we can sort
by ncalls: